In [19]:
import pandas as pd
import numpy as np

import plotly.offline as py


import warnings
warnings.filterwarnings('ignore')

#Machine Learning



#biblioteca de processamento
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix

#Modelos de ML
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import xgboost as xgb

In [2]:
partidas_apartir_94 = pd.read_csv("data/partidas_apartir_94.csv")

In [3]:
#Criando uma lista com todos os times que disputram a copa do mundo
nome_time = {}
index = 0

for idx, jogo in partidas_apartir_94.iterrows():
  nome = jogo['TimeDaCasa']
  if(nome not in nome_time.keys()):
      nome_time[nome] = index
      index += 1
  nome = jogo['TimeVisitante']
  if(nome not in nome_time.keys()):
    nome_time[nome] = index
    index += 1

In [4]:
nome_time

{'Spain': 0,
 'Korea Republic': 1,
 'Germany': 2,
 'Bolivia': 3,
 'USA': 4,
 'Switzerland': 5,
 'Italy': 6,
 'Republic of Ireland': 7,
 'Colombia': 8,
 'Romania': 9,
 'Belgium': 10,
 'Morocco': 11,
 'Norway': 12,
 'Mexico': 13,
 'Cameroon': 14,
 'Sweden': 15,
 'Netherlands': 16,
 'Saudi Arabia': 17,
 'Brazil': 18,
 'Russia': 19,
 'Argentina': 20,
 'Greece': 21,
 'Nigeria': 22,
 'Bulgaria': 23,
 'Scotland': 24,
 'Chile': 25,
 'Austria': 26,
 'Paraguay': 27,
 'Denmark': 28,
 'France': 29,
 'South Africa': 30,
 'Japan': 31,
 'Jamaica': 32,
 'Croatia': 33,
 'England': 34,
 'Tunisia': 35,
 'Yugoslavia': 36,
 'Senegal': 37,
 'Uruguay': 38,
 'Slovenia': 39,
 'Turkey': 40,
 'Ecuador': 41,
 'China PR': 42,
 'Costa Rica': 43,
 'Poland': 44,
 'Portugal': 45,
 'Trinidad and Tobago': 46,
 "Côte d'Ivoire": 47,
 'Serbia and Montenegro': 48,
 'Angola': 49,
 'Australia': 50,
 'Czech Republic': 51,
 'Ghana': 52,
 'Togo': 53,
 'Ukraine': 54,
 'IR Iran': 55,
 'Algeria': 56,
 'Serbia': 57,
 'New Zealand': 

Transformando a feature vencedor em variavel numérica

0 - empate

1 - TimeDaCasa

2 - TimeVisitante

In [5]:
partidas_apartir_94["Vencedor"].replace(to_replace=["Empate", "TimeDaCasa", "TimeVisitante"],
                                            value = [0, 1, 2], inplace = True)

In [6]:
def substituir_por_id(df):
    df['TimeDaCasa'] = nome_time[df['TimeDaCasa']]
    df['TimeVisitante'] = nome_time[df['TimeVisitante']]
    return df

In [7]:
id_time = partidas_apartir_94.apply(substituir_por_id, axis='columns')

In [8]:
id_time["TimeDaCasa"].unique()


array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22,  9,  1, 13, 15, 17,
       23,  5,  3,  7, 19, 11, 21, 27, 29, 32, 34, 24, 25, 30, 31, 38, 33,
       42, 28, 43, 45, 35, 37, 39, 40, 41, 44, 46, 48, 49, 50, 51, 53, 47,
       52, 54, 56, 57, 58, 61, 59, 60, 55, 62, 63, 65, 64, 66],
      dtype=int64)

Etapa de treinamento

In [11]:
X = id_time.loc[:,['TimeDaCasa', 'TimeVisitante', 'QuantidadeTitulosTimeDaCasa','QuantidadeTitulosVisitante', 
                    "RankTimeDaCasa", "RankTimeVisitante"]]

X = np.array(X).astype('float64')

In [12]:
y = id_time.loc[:,['Vencedor']]
y = np.array(y).astype('int')

In [13]:
y.shape

(447, 1)

In [14]:
y = np.reshape(y,(1,447))
y = y[0]

In [16]:
X,y = shuffle(X,y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [17]:
print(y_train)

[1 2 1 2 1 1 1 0 0 2 0 2 1 0 2 2 1 1 1 1 1 1 2 1 1 2 2 0 1 2 1 2 1 1 2 1 2
 2 1 1 2 2 2 0 1 2 2 1 1 2 1 1 2 1 0 1 2 1 1 2 2 1 2 1 1 2 0 1 1 1 2 2 2 1
 1 1 0 1 2 1 2 1 1 2 2 1 1 2 2 1 0 1 0 1 2 1 1 1 0 2 2 0 2 2 2 1 2 2 1 0 0
 0 1 0 2 0 0 1 1 2 2 0 1 1 1 1 1 0 2 1 0 1 0 1 1 1 2 0 2 2 0 2 1 0 0 1 2 1
 1 1 2 0 1 0 0 2 0 2 1 0 1 2 2 2 0 1 0 1 1 1 0 1 0 2 1 0 1 1 0 2 2 2 2 1 0
 1 1 1 1 0 1 1 1 1 1 0 1 0 2 1 0 1 2 1 2 0 2 1 0 0 1 2 1 2 1 1 2 0 1 0 2 0
 1 0 2 1 1 2 2 1 2 1 2 2 1 2 0 2 1 2 1 0 1 1 2 1 0 0 0 2 0 2 1 1 0 2 0 1 1
 1 1 2 0 1 2 0 2 0 2 1 0 0 1 2 0 2 0 1 2 1 0 0 1 2 2 2 1 0 2 1 1 0 2 0 1 2
 1 1 0 1 1 2 0 1 2 0 0 1 0 1 1 2 0 0 1 2 2 1 0 1 0 0 2 2 1 2 1 0 2 1 1 2 0
 0 0 1 2 1 1 1 1 2 1 0 0 0 2 1 2 0 2 0 1 1 2 0 2]


In [20]:
#Os modelos de regressão que utilizaremos aqui em nosso conjunto de dados. 
models = {"KNN": KNeighborsClassifier(),
    "RandomForest": RandomForestClassifier(),
    "DecisionTree": DecisionTreeClassifier(),
    "SVM": SVC(kernel='rbf', class_weight='balanced', probability=True),
    "XGBoost": xgb.XGBRegressor()}

In [21]:
#Criando uma função para criar cada modelo, fazer o fit e o score
def fit_and_score(models, X_train, X_test, y_train, y_test):
    np.random.seed(123)
    model_scores = {}
    
    for name, model in models.items():
        model.fit(X_train, y_train)
        model_scores[name] = model.score(X_test, y_test)
    return model_scores

In [22]:
#Passando o parametros para a nossa função
model_scores = fit_and_score(models=models, 
                             X_train=X_train, 
                             X_test=X_test, 
                             y_train= y_train, 
                             y_test= y_test)

In [23]:
#Verificando o score de cada modelo
model_scores

{'KNN': 0.4888888888888889,
 'RandomForest': 0.5666666666666667,
 'DecisionTree': 0.4666666666666667,
 'SVM': 0.4444444444444444,
 'XGBoost': -0.2072834896412803}

In [24]:
#Criando um dataframe com os retornos de cada modelo
com_model = pd.DataFrame(list(model_scores.items()),
                         columns = ["Model", "Accuracy"])

com_model

,Model,Accuracy
0,KNN,0.488889
1,RandomForest,0.566667
2,DecisionTree,0.466667
3,SVM,0.444444
4,XGBoost,-0.207283


In [25]:
import plotly.express as px
fig = px.bar(com_model, 
             x = "Model", 
             y = "Accuracy",
             title = "Accuracy of each model")
fig.show()

In [26]:
randomforest = RandomForestClassifier()
randomforest.fit(X_train, y_train)

RandomForestClassifier()

In [27]:
print("Predicting on the test set")
y_pred = randomforest.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred, labels=range(3)))

Predicting on the test set
              precision    recall  f1-score   support

           0       0.31      0.31      0.31        16
           1       0.67      0.67      0.67        43
           2       0.55      0.55      0.55        31

    accuracy                           0.57        90
   macro avg       0.51      0.51      0.51        90
weighted avg       0.57      0.57      0.57        90

[[ 5  6  5]
 [ 5 29  9]
 [ 6  8 17]]


In [28]:
ranking_fifa_eve_cup = pd.read_csv("data/ranking_fifa_eve_cup.csv")

In [29]:
dados_2022 = ranking_fifa_eve_cup[ranking_fifa_eve_cup["rank_year"] == 2022]
dados_2022.head()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date,rank_month,rank_year,rank_day
1934,2,Belgium,BEL,1821.92,1827.00,0,UEFA,2022-06-23,6,2022,23
1935,211,San Marino,SMR,763.82,776.97,0,UEFA,2022-06-23,6,2022,23
1936,78,China PR,CHN,1304.02,1304.02,1,AFC,2022-06-23,6,2022,23
1937,77,Uzbekistan,UZB,1306.63,1286.55,-6,AFC,2022-06-23,6,2022,23
1938,76,Israel,ISR,1316.35,1305.92,0,UEFA,2022-06-23,6,2022,23


In [31]:
campeoes = pd.read_csv("data/campeoes_edited.csv")

In [34]:
def predicao(timeA, timeB):
  idA = nome_time[timeA]
  idB = nome_time[timeB]
  campeaoA = campeoes.get(timeA) if campeoes.get(timeA) != None else 0
  campeaoB = campeoes.get(timeB) if campeoes.get(timeB) != None else 0
  rankA = dados_2022[dados_2022["country_full"] == timeA]["rank"].values
  rankB = dados_2022[dados_2022["country_full"] == timeB]["rank"].values
  

  x = np.array([idA, idB, campeaoA, campeaoB, rankA, rankB]).astype('float64')
  x = np.reshape(x, (1,-1))
  _y = randomforest.predict_proba(x)[0]

  text = ('Chance de ' +timeA+' vencer '+timeB+' é {}\nChance de '+timeB+' vencer '+timeA+' e {}\nChance de '+timeA+' e '+timeB+' empatar é {}').format(_y[1]*100,_y[2]*100,_y[0]*100)
  return _y[0], text

In [41]:
prob1, text1 = predicao('Brazil', "Uruguay")
print(text1)

Chance de Brazil vencer Uruguay é 48.0
Chance de Uruguay vencer Brazil e 40.0
Chance de Brazil e Uruguay empatar é 12.0


In [42]:
import joblib

In [44]:
joblib.dump(randomforest, "model.pkl")

['model.pkl']